In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/3.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/5.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112257047,ow2xPaj,0
1,1111920714,LNKXqJB,0
2,1112346945,8MBPqYD,0
3,1112345547,ek4P2Wr,0
4,1112237522,NpqMD5,0
5,1112277633,2zQaobV,0
6,1112302347,JBxO4kk,0
7,1112291495,VNxEXQl,0
8,1112304011,MV6k00x,0
9,1112326289,A3XPKjL,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16459027


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
13186907,1112410021,ZDPw88b,1,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
10536623,1112439473,akDd42E,0,FEM,43.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
9172806,1112402792,owjEDek,0,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,True,NaN
10876795,1112433160,YROv5X,0,FEM,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
7280385,1112325280,kP64BPj,1,FEM,25.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,NaN
7214594,1112208757,A3XGvA5,0,MASC,43.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,True,NaN
3170248,1112454316,kP6OKWZ,1,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,False,False,0.003125
7157143,1112308923,wVk9LwZ,0,FEM,22.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
11234153,1112425472,LN8Qjqr,0,FEM,23.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,True,False,NaN
9308808,1112266098,aNDPWW,0,MASC,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
351518,1112377249,Ebvlq6,0,FEM,27.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,False,False,False,0.000000
4558849,1112417329,MVPRXYN,0,FEM,33.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.000000
5594928,1112296155,axPlZ,0,MASC,44.0,-,Graduado,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,0.000000
7527491,1112426116,Nz0vpN5,1,MASC,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.052632
12094234,1111916200,MVPVjbK,0,FEM,43.0,-,-,Abandonado,-,-,...,False,False,False,False,False,True,False,True,False,0.000000
2512392,1112296411,W9PAWql,1,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
8293619,1112366188,VNkjDNm,1,FEM,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
7788816,1112441146,6r261ov,1,FEM,43.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.005900
12858810,1112273835,bOZkwr2,0,MASC,53.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,False,0.000000
15598659,1111826761,GNJWpNA,0,FEM,27.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,True,False,0.000000


In [15]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  14847370 Test:  781441


In [16]:
gc.collect()

156

In [17]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [18]:
score=c.score(x_test,y_test)*100
print(score)

75.83426004010539


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV13.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV13.pkl']

In [20]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
73187,19313,1112224020,62bkQM,MASC,37.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
43054,6391,1111644641,ZDaN2V1,MASC,41.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
55948,5650,1111570518,YobqGw,MASC,42.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
71731,55288,1112447452,8Ma1Jez,FEM,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.000000
24791,15523,1112141215,YMZoVr,FEM,35.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.008032
34304,40850,1112420185,YmrdBX,FEM,27.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.012346
59538,21442,1112277683,56WRQq,FEM,43.0,-,-,-,Graduado,Graduado,...,False,False,False,False,True,True,False,False,False,0.018182
37181,57050,1112451030,1kG30z,MASC,41.0,-,-,-,En Curso,-,...,False,False,False,False,False,False,False,False,False,0.000000
13557,60593,1112455069,A3XlAwm,MASC,22.0,-,-,-,-,En Curso,...,False,False,False,False,False,False,False,False,False,0.000000
23969,72084,1112463045,Drz33ZZ,MASC,24.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.007143


In [21]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.71892107, 0.28107893],
       [0.72932431, 0.27067569],
       [0.7319109 , 0.2680891 ],
       ...,
       [0.93413585, 0.06586415],
       [0.80726096, 0.19273904],
       [0.9720771 , 0.0279229 ]])

In [22]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [23]:
prediccion['sepostulo']=y_final2

In [24]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.281079
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.270676
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.268089
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.082225
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.053949


In [25]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.281079
1,1,0.270676
2,2,0.268089
3,3,0.082225
4,4,0.053949


In [26]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion28.csv",index=False)

In [27]:
prediccion.sepostulo.mean()

0.5207650106639929